In [1]:
import sqlite3
con = sqlite3.connect("movies.db")
cur = con.cursor()

In [2]:
from enum import Enum
class Country(Enum):
    BRITAIN = 'Britain'
    NEWZEALAND = 'NewZealand'
    USA = 'USA'
    BRAZIL = 'Brazil'

    @classmethod
    def valueTuples(self):
        return [(e.value,) for e in Country]

    @classmethod
    def createTable(self):
        cur.execute('CREATE TABLE IF NOT EXISTS l_countries (country TEXT PRIMARY KEY)')
        cur.executemany('INSERT INTO l_countries VALUES(?)', Country.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS l_countries')
        con.commit()
        
    @classmethod
    def getAll(self):
        cur.execute('SELECT * FROM l_countries')
        sqlResult = cur.fetchall()
        return [Country(p[0]) for p in sqlResult]

In [3]:
Country.removeTable()
Country.createTable()

In [4]:
Country.getAll()

[<Country.BRITAIN: 'Britain'>,
 <Country.NEWZEALAND: 'NewZealand'>,
 <Country.USA: 'USA'>,
 <Country.BRAZIL: 'Brazil'>]

In [5]:
class Genre(Enum):
    ADVENTURE = 'adventure'
    SCIFY = 'scify'
    ACTION = 'action'
    COMEDY = 'comedy'
    HORROR = 'horror'
    OTHERS = 'others'
    
    @classmethod
    def valueTuples(self):
        return [(e.value,) for e in Genre]

    @classmethod
    def createTable(self):
        cur.execute('CREATE TABLE IF NOT EXISTS l_genres (genre text)')
        cur.executemany('INSERT INTO l_genres VALUES(?)', Genre.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS l_genres')
        con.commit()
        
    @classmethod
    def getAll(self):
        cur.execute('SELECT * FROM l_genres')
        sqlResult = cur.fetchall()
        return [Genre(p[0]) for p in sqlResult]

In [6]:
Genre.removeTable()
Genre.createTable()
Genre.getAll()

[<Genre.ADVENTURE: 'adventure'>,
 <Genre.SCIFY: 'scify'>,
 <Genre.ACTION: 'action'>,
 <Genre.COMEDY: 'comedy'>,
 <Genre.HORROR: 'horror'>,
 <Genre.OTHERS: 'others'>]

In [7]:
def createGenreMapping():
    cur.execute('''CREATE TABLE IF NOT EXISTS MOVIE_GENRE(
                    movie text,
                    genre text,
                    PRIMARY KEY(movie, genre),
                    FOREIGN KEY(movie) REFERENCES movies(name),
                    FOREIGN KEY(genre) REFERENCES l_genres(genre))''')
    con.commit()

In [8]:
cur.execute('DROP TABLE IF EXISTS MOVIE_GENRE')
con.commit()

In [9]:
createGenreMapping()

In [10]:
class Movie:
    def __init__(self, name, director, country, year, rating, genres):
        self.name = name
        self.director = director
        self.country = country
        self.year = year
        self.rating = rating
        self.genres = genres

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        name text PRIMARY KEY,
                        director text,
                        country text,
                        year integer,
                        rating REAL,
                        FOREIGN KEY(country) REFERENCES l_countries(country))''')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?, ?)''', (self.name, self.director, self.country, self.year, self.rating))
        cur.executemany('''INSERT INTO movie_genre VALUES(?,?)''', [(self.name, m.value) for m in self.genres])
        con.commit()

    def update(self):
        cur.execute('''UPDATE movies SET director=?, country=?, year=?, rating=? WHERE name=?''', (self.director, self.country, self.year, self.rating, self.name))
        con.commit()
        
    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS movies')
        con.commit()
        
    @classmethod
    def get(self, name):
        cur.execute(''' SELECT director, country, year, rating, GROUP_CONCAT(genre, ',') FROM movies
                    LEFT JOIN movie_genre ON movies.name = movie_genre.movie
                    WHERE movies.name=? GROUP BY movies.name''', (name,))
        res = cur.fetchone()
        if res:
            return Movie(name, res[0], res[1], res[2], res[3], getGenresFromString(res[4]))
        else:
            return None

    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.name, self.director, self.country, self.year, self.rating, self.genres)
    

def getGenresFromString(str):
    return {Genre(e) for e in str.split(',')}

In [11]:
Movie.removeTable()
Movie.createTable()

In [12]:
movie_1 = Movie('One Piece', 'Konosude Uda', 'Japan', 1999, 8.9, {Genre.ADVENTURE})
movie_2 = Movie('Blade', 'Steven Norrington', 'LosAngeles', 1998, 7.1, {Genre.HORROR})
movie_3 = Movie('Pulp fiction', 'Quentin Tarantino', 'USA', 1994, 8.9, {Genre.COMEDY})


movie_1.insert()
movie_2.insert()
movie_3.insert()

In [13]:
print(Movie.get('One Piece'))

(One Piece, Konosude Uda, Japan, 1999, 8.9, {<Genre.ADVENTURE: 'adventure'>})


In [14]:
class Director:
    def __init__(self, name, birth_year, country):
        self.name = name
        self.birth_year = birth_year
        self.country = country

    @classmethod
    def createTable(cls):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        birth_year INTEGER,
                        country TEXT)''')
        con.commit()

    @classmethod
    def insert(self):
        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (self.name, self.birth_year, self.country))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET birth_year=?, country=? WHERE name=?''', (self.birth_year, self.country, self.name))
        con.commit() 
        
    def removeTable(cls):
        cur.execute('DROP TABLE IF EXISTS directors')
        con.commit()


    @classmethod
    def get(self, name):
        cur.execute('SELECT birth_year, country FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        if res:
            return Director(name, res[0], res[1])
        else:
            return None

    @classmethod
    def getAll(self):
        cur.execute('SELECT name, birth_year, country FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], el[2]) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.birth_year, self.country)

In [27]:
Director.removeTable()
Director.createTable()

TypeError: Director.removeTable() missing 1 required positional argument: 'cls'

In [20]:
d1 = Director('Konosude Uda', 1966, 'Japan')
d2 = Director('Steven Norrington', 1964, 'GreatBritain')
d3 = Director('Quentin Tarantino', 1963, 'USA')

d1.insert()
d2.insert()
d3.insert()

AttributeError: type object 'Director' has no attribute 'name'

In [21]:
directorFromDb = Director.get('Quentin Tarantino')
print(directorFromDb)

(Quentin Tarantino, 1963, USA)


In [22]:
class Movies:
    def __init__(self, name, director, country, year, rating, genres):
        self.name = name
        self.director = director
        self.country = country
        self.year = year
        self.rating = rating
        self.genres = genres

    @classmethod
    def insert(self):
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?,?)''', (self.name, self.director, self.country, self.year, self.rating))
        cur.executemany('''INSERT INTO movie_genre VALUES(?,?)''', [(self.name, genre.value) for genre in self.genres])
        con.commit()

    def update(self):
        cur.execute('''UPDATE movies SET director=?, country=?, year=?, rating=? WHERE name=?''', 
                    (self.director, self.country, self.year, self.rating, self.name))
        con.commit() 
        
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        name TEXT PRIMARY KEY,
                        director TEXT,
                        country TEXT,
                        year INTEGER,
                        rating REAL,
                        FOREIGN KEY(director) REFERENCES directors(name))''')
        con.commit()


    @classmethod
    def get(self):
        cur.execute('''
                    SELECT m.name, m.director, d.birth_year, d.country, m.country, m.year, m.rating, GROUP_CONCAT(g.genre, ',')
                    FROM movies m
                    LEFT JOIN directors d ON m.director = d.name
                    LEFT JOIN movie_genre g ON m.name = g.movie
                    GROUP BY m.name
                    ''')
        res = cur.fetchall()
        return res
    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.name, self.director, self.country, self.year, self.rating, self.genres)

In [23]:
Movie.removeTable()
Movie.createTable()

cur.execute('DROP TABLE IF EXISTS MOVIE_GENRE')
con.commit()

In [24]:
createGenreMapping()

In [25]:
movie_1 = Movie('One Piece', 'Konosude Uda', 'Japan', 1999, 8.9, {Genre.ADVENTURE})
movie_2 = Movie('Blade', 'Steven Norrington', 'LosAngeles', 1998, 7.1, {Genre.HORROR})
movie_3 = Movie('Pulp fiction', 'Quentin Tarantino', 'USA', 1994, 8.9, {Genre.COMEDY})


movie_1.insert()
movie_2.insert()
movie_3.insert()

In [26]:
print(movie_1.get("Pulp fiction"))

(Pulp fiction, Quentin Tarantino, USA, 1994, 8.9, {<Genre.COMEDY: 'comedy'>})


### erDiagram

erDiagram


    movie {
        TEXT name PK
        TEXT director
        TEXT country FK
        INTEGER year
        REAL rating

    }

    l_countries {
        TEXT country PK
    }

    directors {
        TEXT name PK
        INTEGER year
        TEXT country
    }

    l_genres {
        TEXT genre PK
    }

    movie_genres {
        TEXT movie FK
        TEXT genre FK
    }


    movie }|--|| directors : "made by"
    movie }|--|| l_countries : from
    movie }|--|{ movie_genres  : "is"
    l_genres||--|{ movie_genres : refers